In [1]:
import pandas as pd 
import numpy as np

# vote_rg (race/gender) 

In [2]:
df_vote_rg = pd.read_csv('ga_n_voted_race_gender.csv')

In [3]:
df_poll = pd.read_excel('ga_polling.xlsx')
df_poll

County  Vote Center ID County Precinct  Municipal Precinct  \
0     APPLING            1012               2                 NaN   
1     APPLING            1018              1C                 NaN   
2     APPLING            1001              1B                 NaN   
3     APPLING            1006              4D                 NaN   
4     APPLING            1002              5A                 NaN   
...       ...             ...             ...                 ...   
2673    WORTH          159014               5                 NaN   
2674    WORTH          159022               4                 NaN   
2675    WORTH          159016              14                 NaN   
2676    WORTH          159017               9                 NaN   
2677    WORTH          159009              15                 NaN   

                    Polling Place Name Street Number  \
0     LIONS CLUB BLDG/ AT FAIR GROUNDS           245   
1           1ST ASSEMBLY OF GOD CHURCH          3397   
2                ALTAMAHA FIRE STATION           392   
3        BAX CH OF GOD/FELLOWSHIP HALL           353   
4                      BAXLEY CITY GYM           252   
...                                ...           ...   
2673             SHINGLER FIRE STATION           126   
2674          SUMNER MUNICIPAL COMPLEX           702   
2675         VOLUNTEER FIRE DEPARTMENT          6104   
2676               WARWICK COMM CENTER           111   
2677          WORTH COUNTY AG BUILDING           102   

                   Street Name       City State   Zip5  Zip4  \
0                INDUSTRIAL DR     BAXLEY    GA  31513   0.0   
1                  HATCH PKY N     BAXLEY    GA  31513   0.0   
2           ALTAMAHA SCHOOL RD     BAXLEY    GA  31513   0.0   
3               BLACKSHEAR HWY     BAXLEY    GA  31513   0.0   
4                W. PARKER ST.     BAXLEY    GA  31513   0.0   
...                        ...        ...   ...    ...   ...   
2673  SHINGLER LITTLE RIVER RD     POULAN    GA  31781   0.0   
2674                 WALNUT ST     SUMNER    GA  31789   0.0   
2675           WILLOW/BIRCH RD     ALBANY    GA  31705   0.0   
2676             DOGWOOD ST SW    WARWICK    GA  31796   0.0   
2677             REBECCA DRIVE  SYLVESTER    GA  31791   0.0   

     Advanced Polling Place Polling Place Status   Classification  
0                        No            Available            OTHER  
1                        No            Available           CHURCH  
2                        No            Available  County Building  
3                        No            Available           CHURCH  
4                        No            Available  County Building  
...                     ...                  ...              ...  
2673                    Yes            Available              NaN  
2674                    Yes            Available              NaN  
2675                    Yes            Available              NaN  
2676                    Yes            Available              NaN  
2677                    Yes            Available              NaN  

[2678 rows x 14 columns]

In [80]:
# df_vote_rg.drop(columns='COUNTY CODE')
df_vote_rg.columns

# new column name, map each race & gender cateogry to A,B,C...; a,b,c
races = ['A', 'B', 'C', 'D', 'E', 'F', 'G']  # 7 races
genders = ['a', 'b', 'c']  # 3 genders

# Generate the list using a list comprehension
column_names = [f"voted_{race}_{gender}" for race in races for gender in genders]
        
renamed_columns = ['county', 'COUNTY CODE'] + column_names + ['total_votes']
df_vote_rg.columns = renamed_columns 
df_vote_rg.drop(columns='COUNTY CODE',inplace=True)
df_vote_rg.head()


county  voted_A_a  voted_A_b  voted_A_c  voted_B_a  voted_B_b  voted_B_c  \
0   APPLING        502        805          0       3045       3534          3   
1  ATKINSON        234        358          1       1062       1246          1   
2     BACON        132        269          0       1845       2164          2   
3     BAKER        237        361          0        440        451          0   
4   BALDWIN       2564       4310          3       4885       5601         10   

   voted_C_a  voted_C_b  voted_C_c  ...  voted_E_a  voted_E_b  voted_E_c  \
0         17         18          0  ...          7          2          0   
1          2          1          0  ...          1          2          0   
2          5          6          0  ...          1          2          0   
3          4          6          0  ...          1          1          0   
4         61         88          0  ...          8          8          0   

   voted_F_a  voted_F_b  voted_F_c  voted_G_a  voted_G_b  voted_G_c  \
0         11         23          0        166        160          1   
1          6          6          0         33         52          1   
2          5         12          0         97         86          0   
3          1          3          0         17         29          0   
4         42         62          0        258        297          2   

   total_votes  
0         8403  
1         3167  
2         4674  
3         1561  
4        18342  

[5 rows x 23 columns]

In [81]:
# df_vote_rg.to_csv('outcome-df/df_n_vote_rg.csv')

In [82]:
# n_sat_act_A_m => number of votes in each racial+gender group 
# df_vote_rg.to_csv('outcome-df/df_n_vote_rg.csv')

# n_sat_act_A => number of votes in each racial group 
# df_vote_r.to_csv('outcome-df/df_n_vote_r.csv')

In [85]:
# Melt the DataFrame to long format for easier aggregation
df_long = df_vote_rg.drop(columns='total_votes').melt(id_vars=['county'], var_name='vote_type', value_name='votes')

# Extract the capital letter from the `vote_type` column
df_long['group'] = df_long['vote_type'].str.extract(r'voted_([A-Z])')

# Group by `county` and `group` and sum the votes
df_summary = df_long.groupby(['county', 'group'])['votes'].sum().reset_index()

# Pivot back to wide format
df_result = df_summary.pivot(index='county', columns='group', values='votes').reset_index()

# Rename the columns for clarity
df_result.columns.name = None  # Remove multi-index column name
df_result.rename(columns=lambda x: f"voted_{x}" if x != 'county' else x, inplace=True)

df_vote_r = df_result
df_vote_r.head()

county  voted_A  voted_B  voted_C  voted_D  voted_E  voted_F  voted_G
0   APPLING     1307     6582       35      109        9       34      327
1  ATKINSON      593     2309        3      161        3       12       86
2     BACON      401     4011       11       48        3       17      183
3     BAKER      598      891       10       10        2        4       46
4   BALDWIN     6877    10496      149      143       16      104      557

In [86]:
# df_vote_r.to_csv('outcome-df/df_n_vote_r.csv')

# Regiseterd people (race/gender)

In [ ]:
# n_A_m => number of eligible people in each racial+gender group
#  df_reg_rg.to_csv('outcome-df/df_reg_rg.csv')

# n_A .. n_G => number of eligible people in each racial group
# df_reg_r.to_csv('outcome-df/df_reg_r.csv')

In [177]:
df_reg_rg = pd.read_csv('ga_n_registered_race_gender.csv')

In [178]:
column_names = [f"registered_{race}_{gender}" for race in races for gender in genders]
renamed_columns = ['county'] + column_names + ['total_registers']
df_reg_rg .columns = renamed_columns 
df_reg_rg

county  registered_A_a  registered_A_b  registered_A_c  \
0      APPLING             967            1195               0   
1     ATKINSON             400             512               1   
2        BACON             340             559               1   
3        BAKER             412             517               0   
4      BALDWIN            4601            6382               5   
..         ...             ...             ...             ...   
154  WHITFIELD            1040            1159               1   
155     WILCOX             570             748               0   
156     WILKES            1147            1529               6   
157  WILKINSON            1151            1456               4   
158      WORTH            1555            2004               4   

     registered_B_a  registered_B_b  registered_B_c  registered_C_a  \
0              4077            4618               4              30   
1              1537            1745               1               2   
2              2543            2905               5               8   
3               575             621               0               5   
4              6592            7489              14             110   
..              ...             ...             ...             ...   
154           17949           20504              52             238   
155            1530            1692               1               9   
156            1894            2016               1              18   
157            1804            1910               3               4   
158            4769            5092               8              10   

     registered_C_b  registered_C_c  ...  registered_E_a  registered_E_b  \
0                30               0  ...              10               5   
1                 3               0  ...               2               7   
2                10               0  ...               3               3   
3                 7               0  ...               1               1   
4               141               0  ...              18              15   
..              ...             ...  ...             ...             ...   
154             289               1  ...              51              39   
155              11               0  ...               4               1   
156              17               0  ...               7               7   
157              11               0  ...               4               6   
158              24               0  ...               6               9   

     registered_E_c  registered_F_a  registered_F_b  registered_F_c  \
0                 0              31              54               0   
1                 0              16              18               0   
2                 0              18              22               0   
3                 0               5               7               0   
4                 1              85             120               0   
..              ...             ...             ...             ...   
154               0             316             363               2   
155               0              15              13               0   
156               0              20              35               0   
157               0              17              22               0   
158               0              45              40               0   

     registered_G_a  registered_G_b  registered_G_c  total_registers  
0               429             375               3            12078  
1               120             134               3             4878  
2               235             188               1             6952  
3                61              51               0             2286  
4               772             661               4            27246  
..              ...             ...             ...              ...  
154            2672            2469              21            55945  
155              72         

In [179]:
# df_reg_rg.to_csv('outcome-df/df_n_reg_rg.csv')

In [107]:
# Melt the DataFrame to long format for easier aggregation
df_long = df_reg_rg.drop(columns='total_registers').melt(id_vars=['county'], var_name='register_type', value_name='registers')

# Extract the capital letter from the `vote_type` column
df_long['group'] = df_long['register_type'].str.extract(r'registered_([A-Z])')

# Group by `county` and `group` and sum the votes
df_summary = df_long.groupby(['county', 'group'])['registers'].sum().reset_index()

# Pivot back to wide format
df_result = df_summary.pivot(index='county', columns='group', values='registers').reset_index()

# Rename the columns for clarity
df_result.columns.name = None  # Remove multi-index column name
df_result.rename(columns=lambda x: f"registered_{x}" if x != 'county' else x, inplace=True)

df_reg_r = df_result
df_reg_r.head()

county  registered_A  registered_B  registered_C  registered_D  \
0   APPLING          2162          8699            60           250   
1  ATKINSON           913          3283             5           377   
2     BACON           900          5453            18           111   
3     BAKER           929          1196            12            23   
4   BALDWIN         10988         14095           251           236   

   registered_E  registered_F  registered_G  
0            15            85           807  
1             9            34           257  
2             6            40           424  
3             2            12           112  
4            34           205          1437

In [108]:
# df_reg_r.to_csv('outcome-df/df_n_reg_r.csv')

# Fractions

In [ ]:
# frac_sat_act_A_m => n_sat_act_A_m /n_A_m ; votes in racial+gender group/number of eligible people in racial+gender group 
# df_fr_vote_rg.to_csv('outcome-df/df_fr_vote_rg.csv')

# frac_sat_act_A => n_sat_act_A /n_sat_act ; votes in racial group/total number of votes
# df_vote_r.to_csv('outcome-df/df_vote_r.csv')


In [120]:
df_fr_vote_rg = pd.read_csv('ga_fr_voted_rg.csv')

In [123]:
# turn into percentage % 
df_fr_vote_rg.loc[:, df_fr_vote_rg.columns != 'county'] = df_fr_vote_rg.loc[:, df_fr_vote_rg.columns != 'county'] / 100

# rename columns
column_names = [f"fr_vote_{race}_{gender}" for race in races for gender in genders]
renamed_columns = ['county'] + column_names
df_fr_vote_rg.columns = renamed_columns 

In [126]:
# df_fr_vote_rg.to_csv('outcome-df/df_fr_vote_rg.csv')

In [156]:
df_vote_r = pd.read_csv('outcome-df/df_n_vote_r.csv').iloc[:,1:]
df_vote_rg = pd.read_csv('outcome-df/df_n_vote_rg.csv')
df_vote_r

county  voted_A  voted_B  voted_C  voted_D  voted_E  voted_F  voted_G
0      APPLING     1307     6582       35      109        9       34      327
1     ATKINSON      593     2309        3      161        3       12       86
2        BACON      401     4011       11       48        3       17      183
3        BAKER      598      891       10       10        2        4       46
4      BALDWIN     6877    10496      149      143       16      104      557
..         ...      ...      ...      ...      ...      ...      ...      ...
154  WHITFIELD     1284    27684      353     4715       51      362     2506
155     WILCOX      759     2439        8       24        2       12       50
156     WILKES     1733     3042       25       22        7       29      166
157  WILKINSON     1832     2777        8       10        7       18      124
158      WORTH     1978     6959       20       41        5       48      271

[159 rows x 8 columns]

In [157]:
df_vote_r.iloc[:, 1:] = df_vote_r.iloc[:, 1:].div(df_vote_rg['total_votes'], axis=0)
df_vote_r

county   voted_A   voted_B   voted_C   voted_D   voted_E   voted_F  \
0      APPLING  0.155540  0.783292  0.004165  0.012972  0.001071  0.004046   
1     ATKINSON  0.187243  0.729081  0.000947  0.050837  0.000947  0.003789   
2        BACON  0.085794  0.858151  0.002353  0.010270  0.000642  0.003637   
3        BAKER  0.383088  0.570788  0.006406  0.006406  0.001281  0.002562   
4      BALDWIN  0.374932  0.572239  0.008123  0.007796  0.000872  0.005670   
..         ...       ...       ...       ...       ...       ...       ...   
154  WHITFIELD  0.034745  0.749127  0.009552  0.127588  0.001380  0.009796   
155     WILCOX  0.230419  0.740437  0.002429  0.007286  0.000607  0.003643   
156     WILKES  0.344944  0.605494  0.004976  0.004379  0.001393  0.005772   
157  WILKINSON  0.383585  0.581449  0.001675  0.002094  0.001466  0.003769   
158      WORTH  0.212186  0.746514  0.002145  0.004398  0.000536  0.005149   

      voted_G  
0    0.038915  
1    0.027155  
2    0.039153  
3    0.029468  
4    0.030367  
..        ...  
154  0.067812  
155  0.015179  
156  0.033041  
157  0.025963  
158  0.029071  

[159 rows x 8 columns]

In [158]:
# df_vote_r.to_csv('outcome-df/df_fr_vote_r.csv')

In [ ]:
# frac_A_a ... frac_G_c: number of eligible people each racial+gender group/total number of elegible people
# df_fr_reg_rg.to_csv('outcome-df/df_fr_reg_rg.csv')

# frac_A ... frac_G: number of eligible people each racial group/total number of elegible people
# df_fr_reg_r.to_csv('outcome-df/df_fr_reg_r.csv')

In [166]:
df_fr_reg_rg = pd.read_csv('outcome-df/df_n_reg_rg.csv').iloc[:,1:]
df_fr_reg_rg.iloc[:,1:]=df_fr_reg_rg.iloc[:,1:].div(df_fr_reg_rg['total_registers'], axis = 0)
df_fr_reg_rg.drop(columns='total_registers',inplace = True)

In [169]:
# df_fr_reg_rg.to_csv('outcome-df/df_fr_reg_rg.csv')

In [182]:
df_fr_reg_r = pd.read_csv('outcome-df/df_n_reg_r.csv').iloc[:,1:]
df_n_reg_rg = pd.read_csv('outcome-df/df_n_reg_rg.csv')

df_fr_reg_r.iloc[:,1:]= df_fr_reg_r.iloc[:,1:].div(df_n_reg_rg['total_registers'], axis = 0)
df_fr_reg_r

county  registered_A  registered_B  registered_C  registered_D  \
0      APPLING      0.179003      0.720235      0.004968      0.020699   
1     ATKINSON      0.187167      0.673022      0.001025      0.077286   
2        BACON      0.129459      0.784379      0.002589      0.015967   
3        BAKER      0.406387      0.523185      0.005249      0.010061   
4      BALDWIN      0.403289      0.517324      0.009212      0.008662   
..         ...           ...           ...           ...           ...   
154  WHITFIELD      0.039324      0.688265      0.009438      0.156922   
155     WILCOX      0.276310      0.675681      0.004193      0.010273   
156     WILKES      0.375683      0.547836      0.004903      0.006864   
157  WILKINSON      0.389005      0.553784      0.002235      0.003278   
158      WORTH      0.246609      0.683070      0.002353      0.006022   

     registered_E  registered_F  registered_G  
0        0.001242      0.007038      0.066816  
1        0.001845      0.006970      0.052686  
2        0.000863      0.005754      0.060990  
3        0.000875      0.005249      0.048994  
4        0.001248      0.007524      0.052742  
..            ...           ...           ...  
154      0.001609      0.012173      0.092269  
155      0.001048      0.005870      0.026625  
156      0.001961      0.007704      0.055050  
157      0.001490      0.005810      0.044398  
158      0.001038      0.005883      0.055025  

[159 rows x 8 columns]

In [184]:
# df_fr_reg_r.to_csv('outcome-df/df_fr_reg_r.csv')

# Poll Data Cleaning 

In [17]:
df_poll.head()

County  Vote Center ID County Precinct  Municipal Precinct  \
0  APPLING            1012               2                 NaN   
1  APPLING            1018              1C                 NaN   
2  APPLING            1001              1B                 NaN   
3  APPLING            1006              4D                 NaN   
4  APPLING            1002              5A                 NaN   

                 Polling Place Name Street Number         Street Name    City  \
0  LIONS CLUB BLDG/ AT FAIR GROUNDS           245       INDUSTRIAL DR  BAXLEY   
1        1ST ASSEMBLY OF GOD CHURCH          3397         HATCH PKY N  BAXLEY   
2             ALTAMAHA FIRE STATION           392  ALTAMAHA SCHOOL RD  BAXLEY   
3     BAX CH OF GOD/FELLOWSHIP HALL           353      BLACKSHEAR HWY  BAXLEY   
4                   BAXLEY CITY GYM           252       W. PARKER ST.  BAXLEY   

  State   Zip5  Zip4 Advanced Polling Place Polling Place Status  \
0    GA  31513   0.0                     No            Available   
1    GA  31513   0.0                     No            Available   
2    GA  31513   0.0                     No            Available   
3    GA  31513   0.0                     No            Available   
4    GA  31513   0.0                     No            Available   

    Classification  
0            OTHER  
1           CHURCH  
2  County Building  
3           CHURCH  
4  County Building

In [18]:
df_poll = df_poll[['County','City']].groupby('County').count().reset_index().rename(columns={'City':'n_poll'})

In [44]:
# df_poll.to_csv('outcome-df/df_poll.csv')

In [3]:
df_poll = pd.read_csv('outcome-df/df_poll.csv')

# tweet data

In [229]:
# extract and save as a smaller csv
# df_tweets_biden = pd.read_csv('tweets/biden.csv', lineterminator='\n', parse_dates=True)
# df_tweets_trump = pd.read_csv('tweets/biden.csv', lineterminator='\n', parse_dates=True) 
# df_tweets_biden = df_tweets_biden[df_tweets_biden['state'] == 'Georgia'][['tweet_id','city','state','lat','long']]
# df_tweets_trump = df_tweets_trump[df_tweets_trump['state'] == 'Georgia'][['tweet_id','city','state','lat','long']]
# df_tweets = pd.concat([df_tweets_biden, df_tweets_trump])
# df_tweets.to_csv('tweets.csv', index=False)

In [4]:
df_tweets = pd.read_csv('tweets.csv')
df_tweets.head()

tweet_id     city    state        lat       long
0  1.316531e+18  Atlanta  Georgia  33.749099 -84.390185
1  1.316534e+18  Atlanta  Georgia  33.749099 -84.390185
2  1.316535e+18  Atlanta  Georgia  33.749099 -84.390185
3  1.316537e+18  Atlanta  Georgia  33.749099 -84.390185
4  1.316542e+18  Atlanta  Georgia  33.749099 -84.390185

In [6]:
# separate into city known; city na 
df_tweets_na = df_tweets[df_tweets['city'].isnull()]
df_tweets_a = df_tweets[df_tweets['city'].notna()]

In [6]:
# ! pip install geopy

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.5 MB/s eta 0:00:00


In [7]:
# for city na- match using lat,long

#!pip install geopy
from geopy.geocoders import Nominatim

# Initialize geolocator
geolocator = Nominatim(user_agent="geoapi")

def get_county(lat, lon):
    try:
        location = geolocator.reverse((lat, lon))
        if location:
            address = location.raw.get('address', {})
            county = address.get('county', 'Unknown')
            return county
        return 'Unknown'
    except Exception as e:
        return 'Unknown'
        
# Apply function to rows with missing city
df_tweets_na['county'] = df_tweets_na.apply(
    lambda row: get_county(row['lat'], row['long']) if pd.isnull(row['city']) else row['county'], axis=1
)


/tmp/ipykernel_615/3557080430.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets_na['county'] = df_tweets_na.apply(


In [8]:
df_tweets_na.head()

tweet_id city    state        lat       long         county
7   1.316550e+18  NaN  Georgia  33.892176 -84.298830  DeKalb County
10  1.316557e+18  NaN  Georgia  33.715670 -83.798790  Walton County
14  1.316567e+18  NaN  Georgia  32.329381 -83.113737   Dodge County
21  1.316651e+18  NaN  Georgia  34.065933 -84.676880    Cobb County
25  1.316707e+18  NaN  Georgia  34.165097 -84.799938  Bartow County

In [9]:
df_tweets_na['county'].isnull().sum()

0

In [15]:
df_tweets_na = df_tweets_na[['tweet_id', 'county']]
# df_tweets_na.to_csv('df_tweets_na.csv')

In [4]:
df_tweets_na = pd.read_csv('df_tweets_na.csv')[['tweet_id','county']]
df_tweets_na

tweet_id         county
0     1.316550e+18  DeKalb County
1     1.316557e+18  Walton County
2     1.316567e+18   Dodge County
3     1.316651e+18    Cobb County
4     1.316707e+18  Bartow County
...            ...            ...
1077  1.325284e+18    Cobb County
1078  1.325419e+18    Cobb County
1079  1.325420e+18    Cobb County
1080  1.325420e+18    Cobb County
1081  1.325420e+18    Cobb County

[1082 rows x 2 columns]

In [32]:
df_tweets = df_tweets_na.groupby('county').count().reset_index()
df_tweets.columns = ['county','tweets']
df_tweets['county'] = df_tweets['county'].str.replace(' County', '', case=False).str.upper()
df_tweets.head()

county  tweets
0  BALDWIN       2
1   BARTOW      49
2  BERRIEN       2
3   BROOKS       8
4  BULLOCH       2

In [33]:
df_tweets.to_csv('df_tweets_count.csv')

In [7]:
# for city known, match using us city,county dataset 
df_cities = pd.read_csv('uscities.csv')
df_cities  = df_cities[df_cities['state_name']=='Georgia'][['city','county_name']]
df_tweets_a =  df_tweets_a.merge(df_cities,on ='city')[['tweet_id','county_name']]

In [8]:
df_tweets_a.columns = ['tweets', 'county']
df_tweets_a = df_tweets_a.groupby('county').count().reset_index()
df_tweets_a

county  tweets
0      Bibb      36
1   Chatham     162
2    Clarke       6
3      Cobb     102
4   Fayette       4
5    Fulton    4902
6   Lowndes       6
7  Richmond      64
8      Tift       8

In [9]:
df_tweets_na = pd.read_csv('df_tweets_count.csv')[['county', 'tweets']]
df_tweets_na

county  tweets
0     BALDWIN       2
1      BARTOW      49
2     BERRIEN       2
3      BROOKS       8
4     BULLOCH       2
5      CAMDEN       8
6     CARROLL       2
7     CHATHAM       2
8   CHATTOOGA      16
9        COBB     347
10     COWETA       2
11       DADE       6
12     DAWSON       2
13     DEKALB      22
14      DODGE      84
15    EMANUEL       2
16     FANNIN       2
17     FULTON      72
18     GILMER       4
19   GWINNETT     154
20     HARRIS       2
21      HENRY     118
22    LAURENS       2
23    LIBERTY       6
24       LONG     104
25    LOWNDES       6
26    LUMPKIN      12
27   ROCKDALE       8
28     TOOMBS       4
29    UNKNOWN       2
30     WALKER      24
31     WALTON       6

In [12]:
df_tweets = pd.concat([df_tweets_a, df_tweets_na])
df_tweets = df_tweets.groupby('county').sum().reset_index()
df_tweets

county  tweets
0     BALDWIN       2
1      BARTOW      49
2     BERRIEN       2
3      BROOKS       8
4     BULLOCH       2
5        Bibb      36
6      CAMDEN       8
7     CARROLL       2
8     CHATHAM       2
9   CHATTOOGA      16
10       COBB     347
11     COWETA       2
12    Chatham     162
13     Clarke       6
14       Cobb     102
15       DADE       6
16     DAWSON       2
17     DEKALB      22
18      DODGE      84
19    EMANUEL       2
20     FANNIN       2
21     FULTON      72
22    Fayette       4
23     Fulton    4902
24     GILMER       4
25   GWINNETT     154
26     HARRIS       2
27      HENRY     118
28    LAURENS       2
29    LIBERTY       6
30       LONG     104
31    LOWNDES       6
32    LUMPKIN      12
33    Lowndes       6
34   ROCKDALE       8
35   Richmond      64
36     TOOMBS       4
37       Tift       8
38    UNKNOWN       2
39     WALKER      24
40     WALTON       6

In [65]:
# df_tweets.to_csv('outcome-df/total_tweets.csv')

In [45]:
df_tweets = pd.read_csv('outcome-df/total_tweets.csv').iloc[:,1:]

In [31]:
df_tweets['county'] = df_tweets['county'].str.replace(' county', '', case=False).str.upper()

In [35]:
df_poll['County']

0        APPLING
1       ATKINSON
2          BACON
3          BAKER
4        BALDWIN
         ...    
154    WHITFIELD
155       WILCOX
156       WILKES
157    WILKINSON
158        WORTH
Name: County, Length: 159, dtype: object

In [46]:
df_tweets = pd.merge(df_poll, df_tweets, left_on='County', right_on='county', how='left')[['County','tweets']]
df_tweets['tweets'] = df_tweets['tweets'].fillna(0)
df_tweets

County  tweets
0      APPLING     0.0
1     ATKINSON     0.0
2        BACON     0.0
3        BAKER     0.0
4      BALDWIN     2.0
..         ...     ...
154  WHITFIELD     0.0
155     WILCOX     0.0
156     WILKES     0.0
157  WILKINSON     0.0
158      WORTH     0.0

[159 rows x 2 columns]

In [50]:
# df_tweets.to_csv('outcome-df/df_tweets.csv')

In [26]:
df_tweets['county'].unique()

array([nan, 'BALDWIN', 'BARTOW', 'BERRIEN', 'BIBB', 'BROOKS', 'BULLOCH',
       'CAMDEN', 'CARROLL', 'CHATHAM', 'CHATTOOGA', 'CLARKE', 'COBB',
       'COWETA', 'DADE', 'DAWSON', 'DEKALB', 'DODGE', 'EMANUEL', 'FANNIN',
       'FAYETTE', 'FULTON', 'GILMER', 'GWINNETT', 'HARRIS', 'HENRY',
       'LAURENS', 'LIBERTY', 'LONG', 'LOWNDES', 'LUMPKIN', 'RICHMOND',
       'ROCKDALE', 'TIFT', 'TOOMBS', 'WALKER', 'WALTON'], dtype=object)

In [25]:
df_poll['County'].unique()

array(['APPLING', 'ATKINSON', 'BACON', 'BAKER', 'BALDWIN', 'BANKS',
       'BARROW', 'BARTOW', 'BEN HILL', 'BERRIEN', 'BIBB', 'BLECKLEY',
       'BRANTLEY', 'BROOKS', 'BRYAN', 'BULLOCH', 'BURKE', 'BUTTS',
       'CALHOUN', 'CAMDEN', 'CANDLER', 'CARROLL', 'CATOOSA', 'CHARLTON',
       'CHATHAM', 'CHATTAHOOCHEE', 'CHATTOOGA', 'CHEROKEE', 'CLARKE',
       'CLAY', 'CLAYTON', 'CLINCH', 'COBB', 'COFFEE', 'COLQUITT',
       'COLUMBIA', 'COOK', 'COWETA', 'CRAWFORD', 'CRISP', 'DADE',
       'DAWSON', 'DECATUR', 'DEKALB', 'DODGE', 'DOOLY', 'DOUGHERTY',
       'DOUGLAS', 'EARLY', 'ECHOLS', 'EFFINGHAM', 'ELBERT', 'EMANUEL',
       'EVANS', 'FANNIN', 'FAYETTE', 'FLOYD', 'FORSYTH', 'FRANKLIN',
       'FULTON', 'GILMER', 'GLASCOCK', 'GLYNN', 'GORDON', 'GRADY',
       'GREENE', 'GWINNETT', 'HABERSHAM', 'HALL', 'HANCOCK', 'HARALSON',
       'HARRIS', 'HART', 'HEARD', 'HENRY', 'HOUSTON', 'IRWIN', 'JACKSON',
       'JASPER', 'JEFF DAVIS', 'JEFFERSON', 'JENKINS', 'JOHNSON', 'JONES',
       'LAMAR', 'LANIE

In [33]:
df_contribution = pd.read_excel('ga_contribution.xlsx')
df_contribution ['County'] = df_contribution ['County'].str.replace(' County', '', case=False).str.upper()
df_contribution.shape

(314, 8)

In [40]:
df_contribution = df_contribution[['County','Total']].groupby('County').sum().reset_index()
df_contribution.columns = ['county','contribution']

In [47]:
# df_contribution.to_csv('outcome-df/df_contribution.csv')

In [67]:
# need columns: 
# ap_ib => total_tweets (county tweets)
# counselors => df_contribution
# calculus => polling availability (need to divide n_poll/total number of elegible people & set a threshold to turn into 0 or 1) 
# frpl_rate? not sure 
# frac_sat_act => total_votes/total number of elegible people


# n_sat_ac => total_votes

# n_A_m => number of eligible people in each racial+gender group
# n_A .. n_G => number of eligible people in each racial group
# frac_A_a ... frac_G_c: number of eligible people each racial+gender group/total number of elegible people
# frac_A ... frac_G: number of eligible people each racial group/total number of elegible people

# n_sat_act_A_m => number of votes in each racial+gender group 
# n_sat_act_A => number of votes in each racial group 
# frac_sat_act_A_m => n_sat_act_A_m /n_A_m ; votes in racial+gender group/number of eligible people in racial+gender group 
# frac_sat_act_A => n_sat_act_A /n_sat_act ; votes in racial group/total number of voters

# total_students => total number of eligible people

# latitude 
# longtitude 

In [129]:
p2_features['frac_sat_act_A_m']

0      0.263473
1      0.259528
2      0.243523
3      0.296296
4      0.272727
         ...   
485    0.000000
486    0.000000
487    0.000000
488    0.000000
489    0.000000
Name: frac_sat_act_A_m, Length: 490, dtype: float64

In [128]:
p2_features['n_sat_act_A_m']/p2_features['n_A_m']

0      0.263473
1      0.259528
2      0.243523
3      0.296296
4      0.272727
         ...   
485         NaN
486         NaN
487         NaN
488         NaN
489         NaN
Length: 490, dtype: float64

In [102]:
p2_features = pd.read_csv('p2_features.csv')
p2_features

Unnamed: 0  ap_ib  calculus  counselors  frpl_rate  frac_sat_act  \
0             0    1.0       1.0        8.48   0.859318      0.269287   
1             1    1.0       1.0        7.20   0.786228      0.291291   
2             2    1.0       0.0        2.13   0.802850      0.203791   
3             3    1.0       0.0        2.12   0.801567      0.275862   
4             4    1.0       1.0        1.08   0.500888      0.410619   
..          ...    ...       ...         ...        ...           ...   
485         485    1.0       0.0        1.32   0.807453      0.261954   
486         486    1.0       1.0        0.00   0.776413      0.165025   
487         487    1.0       0.0        2.07   0.870130      0.137634   
488         488    1.0       1.0        2.19   0.789753      0.241563   
489         489    1.0       0.0        2.17   0.853832      0.263717   

     n_sat_act  n_A_m  n_A_f  n_B_m  ...  frac_sat_act_A  frac_sat_act_B  \
0          370    167    219    429  ...        0.259067        0.260210   
1          582    551    255    235  ...        0.267990        0.258278   
2           86    193    129     23  ...        0.220497        0.122807   
3          104    108    127     58  ...        0.302128        0.211864   
4          232     33     36    141  ...        0.405797        0.391586   
..         ...    ...    ...    ...  ...             ...             ...   
485        126      0    115      0  ...        0.226087        0.276451   
486         67      0    197      0  ...        0.187817        0.104938   
487         64      0    170      0  ...        0.164706        0.121771   
488        136      0    247      0  ...        0.283401        0.136364   
489        149      0     41      0  ...        0.097561        0.237668   

     frac_sat_act_C  frac_sat_act_D  frac_sat_act_E  frac_sat_act_F  \
0          0.206897        0.393443        0.600000        0.428571   
1          0.295739        0.395973        0.500000        0.272727   
2          0.000000        0.400000        0.000000        0.173913   
3          0.142857        0.500000        0.500000        0.666667   
4          0.426087        0.484848        0.000000        0.666667   
..              ...             ...             ...             ...   
485        0.166667        0.282051        0.333333        0.000000   
486        0.285714        0.250000        0.000000        0.166667   
487        0.200000        0.090909        0.000000        0.333333   
488        0.384615        0.217172        0.125000        0.200000   
489        0.340000        0.287234        0.500000        0.000000   

     frac_sat_act_G  total_students   latitude  longitude  
0          0.379310            1374  40.818291 -73.950111  
1          0.133333            1998  40.582372 -73.968078  
2          0.333333             422  40.649485 -73.958213  
3          0.000000             377  40.888012 -73.852561  
4          0.444444             565  40.742723 -73.959366  
..              ...             ...        ...        ...  
485        0.363636             481  40.792917 -73.947165  
486        0.500000             406  40.709800 -73.943942  
487        0.000000             465  40.848581 -73.908670  
488        0.312500             563  40.708447 -73.804211  
489        0.000000             565  40.771307 -73.924286  

[490 rows x 94 columns]

In [103]:
p2_features.columns

Index(['Unnamed: 0', 'ap_ib', 'calculus', 'counselors', 'frpl_rate',
       'frac_sat_act', 'n_sat_act', 'n_A_m', 'n_A_f', 'n_B_m', 'n_B_f',
       'n_C_m', 'n_C_f', 'n_D_m', 'n_D_f', 'n_E_m', 'n_E_f', 'n_F_m', 'n_F_f',
       'n_G_m', 'n_G_f', 'n_A', 'n_B', 'n_C', 'n_D', 'n_E', 'n_F', 'n_G',
       'frac_A_m', 'frac_A_f', 'frac_B_m', 'frac_B_f', 'frac_C_m', 'frac_C_f',
       'frac_D_m', 'frac_D_f', 'frac_E_m', 'frac_E_f', 'frac_F_m', 'frac_F_f',
       'frac_G_m', 'frac_G_f', 'frac_A', 'frac_B', 'frac_C', 'frac_D',
       'frac_E', 'frac_F', 'frac_G', 'n_sat_act_A_m', 'n_sat_act_A_f',
       'n_sat_act_B_m', 'n_sat_act_B_f', 'n_sat_act_C_m', 'n_sat_act_C_f',
       'n_sat_act_D_m', 'n_sat_act_D_f', 'n_sat_act_E_m', 'n_sat_act_E_f',
       'n_sat_act_F_m', 'n_sat_act_F_f', 'n_sat_act_G_m', 'n_sat_act_G_f',
       'n_sat_act_A', 'n_sat_act_B', 'n_sat_act_C', 'n_sat_act_D',
       'n_sat_act_E', 'n_sat_act_F', 'n_sat_act_G', 'frac_sat_act_A_m',
       'frac_sat_act_A_f', 'frac_sat_act_B

In [119]:
p2_features[['frac_A_m', 'frac_A_f', 'frac_B_m', 'frac_B_f', 'frac_C_m', 'frac_C_f',
       'frac_D_m', 'frac_D_f', 'frac_E_m', 'frac_E_f', 'frac_F_m', 'frac_F_f',
       'frac_G_m', 'frac_G_f']].sum(axis=1)

frac_A_m  frac_A_f  frac_B_m  frac_B_f  frac_C_m  frac_C_f  frac_D_m  \
0    0.121543  0.159389  0.312227  0.311499  0.012373  0.008734  0.024017   
1    0.275776  0.127628  0.117618  0.109109  0.117117  0.082583  0.089590   
2    0.457346  0.305687  0.054502  0.080569  0.016588  0.002370  0.011848   
3    0.286472  0.336870  0.153846  0.159151  0.013263  0.005305  0.010610   
4    0.058407  0.063717  0.249558  0.297345  0.111504  0.092035  0.035398   
..        ...       ...       ...       ...       ...       ...       ...   
485  0.000000  0.239085  0.000000  0.609148  0.000000  0.037422  0.000000   
486  0.000000  0.485222  0.000000  0.399015  0.000000  0.051724  0.000000   
487  0.000000  0.365591  0.000000  0.582796  0.000000  0.010753  0.000000   
488  0.000000  0.438721  0.000000  0.117229  0.000000  0.023091  0.000000   
489  0.000000  0.072566  0.000000  0.394690  0.000000  0.176991  0.000000   

     frac_D_f  frac_E_m  frac_E_f  frac_F_m  frac_F_f  frac_G_m  frac_G_f  
0    0.020378  0.002183  0.001456  0.002911  0.002183  0.011645  0.009461  
1    0.059560  0.005005  0.003003  0.004004  0.001502  0.005506  0.002002  
2    0.000000  0.000000  0.002370  0.030806  0.023697  0.009479  0.004739  
3    0.010610  0.002653  0.002653  0.002653  0.005305  0.005305  0.005305  
4    0.023009  0.000000  0.000000  0.001770  0.003540  0.030088  0.033628  
..        ...       ...       ...       ...       ...       ...       ...  
485  0.081081  0.000000  0.006237  0.000000  0.004158  0.000000  0.022869  
486  0.029557  0.000000  0.004926  0.000000  0.014778  0.000000  0.014778  
487  0.023656  0.000000  0.002151  0.000000  0.006452  0.000000  0.008602  
488  0.351687  0.000000  0.014210  0.000000  0.026643  0.000000  0.028419  
489  0.332743  0.000000  0.014159  0.000000  0.007080  0.000000  0.001770  

[490 rows x 14 columns]

In [113]:
p2_features[['n_A_m', 'n_A_f', 'n_B_m', 'n_B_f',
       'n_C_m', 'n_C_f', 'n_D_m', 'n_D_f', 'n_E_m', 'n_E_f', 'n_F_m', 'n_F_f',
       'n_G_m', 'n_G_f']].sum(axis=1)

0      1374
1      1998
2       422
3       377
4       565
       ... 
485     481
486     406
487     465
488     563
489     565
Length: 490, dtype: int64

In [114]:
p2_features['total_students']

0      1374
1      1998
2       422
3       377
4       565
       ... 
485     481
486     406
487     465
488     563
489     565
Name: total_students, Length: 490, dtype: int64

In [118]:
p2_features[['frac_A', 'frac_B', 'frac_C', 'frac_D',
       'frac_E', 'frac_F', 'frac_G']]


frac_A    frac_B    frac_C    frac_D    frac_E    frac_F    frac_G
0    0.280932  0.623726  0.021106  0.044396  0.003639  0.005095  0.021106
1    0.403403  0.226727  0.199700  0.149149  0.008008  0.005506  0.007508
2    0.763033  0.135071  0.018957  0.011848  0.002370  0.054502  0.014218
3    0.623342  0.312997  0.018568  0.021220  0.005305  0.007958  0.010610
4    0.122124  0.546903  0.203540  0.058407  0.000000  0.005310  0.063717
..        ...       ...       ...       ...       ...       ...       ...
485  0.239085  0.609148  0.037422  0.081081  0.006237  0.004158  0.022869
486  0.485222  0.399015  0.051724  0.029557  0.004926  0.014778  0.014778
487  0.365591  0.582796  0.010753  0.023656  0.002151  0.006452  0.008602
488  0.438721  0.117229  0.023091  0.351687  0.014210  0.026643  0.028419
489  0.072566  0.394690  0.176991  0.332743  0.014159  0.007080  0.001770

[490 rows x 7 columns]

In [125]:
p2_features[['n_A_m', 'n_A_f']].sum(axis=1)/p2_features['total_students']

0      0.280932
1      0.403403
2      0.763033
3      0.623342
4      0.122124
         ...   
485    0.239085
486    0.485222
487    0.365591
488    0.438721
489    0.072566
Length: 490, dtype: float64